In [1]:
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import rnn
import math
import numpy as np
import my_txtutils as txt

tf.set_random_seed(0)
SEQLEN = 10
BATCHSIZE = 50

shakedir = "shakespeare/*.txt"
codetext, valitext, bookranges = txt.read_data_files(shakedir, validation=True)

epoch_size = len(codetext) // (BATCHSIZE * SEQLEN)
txt.print_data_stats(len(codetext), len(valitext), epoch_size)

Loading file shakespeare/troilusandcressida.txt
Loading file shakespeare/sonnets.txt
Loading file shakespeare/tempest.txt
Loading file shakespeare/coriolanus.txt
Loading file shakespeare/loverscomplaint.txt
Loading file shakespeare/various.txt
Loading file shakespeare/kingrichardii.txt
Loading file shakespeare/midsummersnightsdream.txt
Loading file shakespeare/kinghenryv.txt
Loading file shakespeare/3kinghenryvi.txt
Loading file shakespeare/twelfthnight.txt
Loading file shakespeare/kingrichardiii.txt
Loading file shakespeare/romeoandjuliet.txt
Loading file shakespeare/measureforemeasure.txt
Loading file shakespeare/comedyoferrors.txt
Loading file shakespeare/muchadoaboutnothing.txt
Loading file shakespeare/titusandronicus.txt
Loading file shakespeare/cymbeline.txt
Loading file shakespeare/loveslabourslost.txt
Loading file shakespeare/juliuscaesar.txt
Loading file shakespeare/antonyandcleopatra.txt
Loading file shakespeare/allswellthatendswell.txt
Loading file shakespeare/2kinghenryvi.t

In [2]:
INTERNALSIZE = 512
NLAYERS = 3

ALPHASIZE = txt.ALPHASIZE

print(len(codetext), len(valitext), len(bookranges))
print(codetext[20], valitext[20], bookranges[20])

5161137 122658 42
35 43 {'start': 2412621, 'end': 2571178, 'name': 'antonyandcleopatra.txt'}


In [3]:

graph = tf.Graph()
with graph.as_default():
    
    Xd = tf.placeholder(tf.uint8, [None, None])  # [ BATCHSIZE, SEQLEN ]
    X = tf.one_hot(Xd, ALPHASIZE, 1.0, 0.0)      # [ BATCHSIZE, SEQLEN, ALPHASIZE ]
    Yd_ = tf.placeholder(tf.uint8, [None, None]) # [ BATCHSIZE, SEQLEN ]
    Y_ = tf.one_hot(Yd_, ALPHASIZE, 1.0, 0.0)    # [ BATCHSIZE, SEQLEN, ALPHASIZE ]

    # input state
    Hin = tf.placeholder(tf.float32, [None, INTERNALSIZE*NLAYERS])  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]

    cells = [tf.nn.rnn_cell.GRUCell(INTERNALSIZE) for _ in range(NLAYERS)]

    mcell = tf.nn.rnn_cell.MultiRNNCell(cells, state_is_tuple=False)

    Hr, H = tf.nn.dynamic_rnn(mcell, X, dtype=tf.float32, initial_state=Hin)

    Hf = tf.reshape(Hr, [-1,INTERNALSIZE])

    Ylogits = layers.linear(Hf, ALPHASIZE)     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]

    Y = tf.nn.softmax(Ylogits)        # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
    Yp = tf.argmax(Y, 1)                          # [ BATCHSIZE x SEQLEN ]
    Yp = tf.reshape(Y, [BATCHSIZE, -1])  # [ BATCHSIZE, SEQLEN ]

    loss = tf.nn.softmax_cross_entropy_with_logits(logits = Ylogits, labels = Y_)
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    # Add variable initializer.
    init = tf.global_variables_initializer()

In [4]:
# init = tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
# sess = tf.Session(config=tf.ConfigProto(gpu_options = gpu_options,
#   allow_soft_placement=True, log_device_placement=True))

with tf.Session(graph=graph, config=config) as sess:
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    for  x, y_, epoch  in txt.rnn_minibatch_sequencer(codetext, BATCHSIZE, SEQLEN, nb_epochs=10):
        print(x.shape,"hahahhaha",y_.shape)
        inH = np.zeros([BATCHSIZE, INTERNALSIZE*NLAYERS])
        # train on one minibatch
        feed_dict = {Xd: x, Yd_: y_, Hin: inH}
        _, y, outH = sess.run([optimizer, Yp, H], feed_dict=feed_dict)
        y = sess.run(Yp, feed_dict=feed_dict)


Initialized
(50, 10) hahahhaha (50, 10)


InternalError: Blas GEMM launch failed : a.shape=(50, 610), b.shape=(610, 1024), m=50, n=1024, k=610
	 [[Node: rnn/while/rnn/multi_rnn_cell/cell_0/cell_0/gru_cell/gates/gates/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](rnn/while/rnn/multi_rnn_cell/cell_0/cell_0/gru_cell/gates/gates/concat, rnn/while/rnn/multi_rnn_cell/cell_0/cell_0/gru_cell/gates/gates/MatMul/Enter)]]
	 [[Node: rnn/while/Exit_2/_9 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2237_rnn/while/Exit_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'rnn/while/rnn/multi_rnn_cell/cell_0/cell_0/gru_cell/gates/gates/MatMul', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, i